# Здесь лежат листинги, которые являются полезными паттернами

In [2]:
# Для красивой визуализации
import seaborn as sns

sns.set(rc={'figure.figsize': (9, 6)})

In [ ]:
# проверим количество пропусков в датафрейме
import pandas as pd

df.isnull().sum()

In [ ]:
# удаляем лишние колонки из датафрейма
X = df.drop(['PassengerId', 'Survived', 'Name', 'Ticket', 'Cabin'], axis=1)
X.head()

In [ ]:
# садим дерево
from sklearn import tree

clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=1000)

In [ ]:
# разобъём выборку на тренировочную и тестовую
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size=0.33,
                                                    random_state=42)

In [ ]:
# проверим результат прогноза
print(f'Тренировочная выборка {np.round(clf.score(X_train, y_train), 4)}')
print(f'Тестовая выборка {np.round(clf.score(X_test, y_test), 4)}')

In [ ]:
# проведём кросс-валидацию
from sklearn.model_selection import cross_val_score

clf = tree.DecisionTreeClassifier(criterion='entropy', max_depth=4)

# протестируем наш классификатор c помощью кросс-валидации
cross_val_score(clf, X_train, y_train, cv=5)

# получим среднюю точность на кросс-валидации
cross_val_score(clf, X_train, y_train, cv=5).mean()

In [ ]:
# садим 100 разных по глубине решающих деревьев
rs = np.random.seed(0)
max_depth_values = range(1, 101)
scores = []

for i in max_depth_values:
    clf = tree.DecisionTreeClassifier(criterion='entropy',
                                     max_depth=i,
                                     random_state=rs)
    
    # обучаем модель и делаем прогноз на тестовых данных
    clf.fit(X_train, y_train)
    scores.append(clf.score(X_test, y_test))

# визуализируем значения score
ax = sns.lineplot(data=scores)
ax.lines[0].set_linestyle('--')
plt.xlabel('Глибина деревьев')
plt.ylabel('Точность скора');

In [3]:
# вложенный генератор списков
x = [1, 2, 3]
y = [4, 5, 6]

c = [[i * j for j in y] for i in x]
print(c)

[[4, 5, 6], [8, 10, 12], [12, 15, 18]]


In [ ]:
# рассмотрим долю пропущенных значений
df.isna().sum() / df.shape[0]

In [ ]:
# заполним пропуски модой
df = df.fillna(value=df.mode().iloc[0])

In [ ]:
# cделаем One-Hot преобразование при помощи pandas
df_label = pd.get_dummies(df, drop_first=True)

In [ ]:
# стандартизируем данные после разбиения
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

In [ ]:
# обучим модель логистической регрессии
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression()
clf.fit(X_train_std, y_train)

In [ ]:
# рассмотрим влияние признаков логистической регрессии на целевую переменную
coef = pd.DataFrame(data=clf.coef_, columns=X.columns)
coef = coef.T
coef.columns = ['coefficient']
coe

In [ ]:
# рассмотрим метрику Accuracy:
from sklearn.metrics import accuracy_score

y_pred_test = clf.predict(X_test_std)
y_pred_train = clf.predict(X_train_std)

print(f'Accuracy Train: {accuracy_score(y_train, y_pred_train):.4f}')
print(f'Accuracy Test: {accuracy_score(y_test, y_pred_test):.4f}')

In [ ]:
# рассмотрим метрики precision (точность) и recall (полнота)
from sklearn.metrics import precision_score, recall_score

print(f'Precision Train: {precision_score(y_train, y_pred_train):.4f}')
print(f'Precision Test: {precision_score(y_test, y_pred_test):.4f}')
print()
print(f'Recall Train: {recall_score(y_train, y_pred_train):.4f}')
print(f'Recall Test: {recall_score(y_test, y_pred_test):.4f}')

In [ ]:
# рассмотрим метрику f1 (среднее гармоническое)
from sklearn.metrics import f1_score

print(f'F1 train: {f1_score(y_train, y_pred_train):.4f}')
print(f'F1 test: {f1_score(y_test, y_pred_test):.4f}')

In [ ]:
# рассмотрим метрику ROC-AUC (площадь под кривой ошибок)
from sklearn.metrics import roc_auc_score

y_pred_test_score = clf.predict_proba(X_test_std)
y_pred_train_score = clf.predict_proba(X_train_std)

print(f'ROC-AUC Train: {roc_auc_score(y_train, y_pred_train_score[:,1]):.4f}')
print(f'ROC-AUC Test: {roc_auc_score(y_test, y_pred_test_score[:,1]):.4f}')

In [7]:
# убираем предупреждения с экрана
import warnings

warnings.filterwarnings('ignore')

# Используем GridSearch при подборе

In [2]:
# импортируем библиотеки
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

In [2]:
# загружаем датасет Титаник
df = sns.load_dataset('titanic')

# удаляем неинформативные колонки
df.drop(columns=['alive', 'deck'], inplace=True)

# заполняем пропуски модами каждой из колонок
df = df.fillna(df.mode().iloc[0])

# разбиваем категориальные столбцы и удаляем один из классов,
# чтобы предобвратить мультиколлинеарность
df_label = pd.get_dummies(df, drop_first=True)

# выделяем X и y
X = df_label.drop('survived', axis=1)
y = df.survived

# разбиваем данные на train и test
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    stratify=y,
                                                    test_size=0.2,
                                                    random_state=42)

# стандартизируем данные
sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

# обучаем модель логистической регрессии
clf = LogisticRegression()
clf.fit(X_train_std, y_train)

# получаем вероятности
y_train_proba = clf.predict_proba(X_train_std)
y_test_proba = clf.predict_proba(X_test_std)

# получаем значение метрики ROC-AUC
print(f'ROC-AUC train = {roc_auc_score(y_train, y_train_proba[:, 1]):.4f}')
print(f'ROC-AUC test = {roc_auc_score(y_test, y_test_proba[:, 1]):.4f}')

ROC-AUC train = 0.8690
ROC-AUC test = 0.8683


In [8]:
warnings.filterwarnings('ignore')

# словарь для перебора
parameters = {
    'max_iter': [1000],
    'C': [1, 10, 100],
    "solver": ["liblinear", "saga", "sag", "lbfgs"],
    'penalty': ['l1', 'l2', 'elasticnet', 'none'],
    "random_state": [42],
    "class_weight" : ['balanced']
}

lr = LogisticRegression()
# подаем на вход модель, словарь с параметрами, установим, на какую метрику ориентироваться

clf = GridSearchCV(estimator=lr, # оценщик (лог рег)
                   param_grid=parameters,
                   scoring='roc_auc',  # метрика
                   cv=3,  # the cross-validation splitting strategy
                   verbose=0)  # отстутствие сообщений
clf.fit(X_train_std, y_train)

# получаем вероятности
y_train_grid = clf.predict_proba(X_train_std)
y_test_grid = clf.predict_proba(X_test_std)

# рассмотрим метрику ROC-AUC
print(f'ROC-AUC train = {roc_auc_score(y_train, y_train_grid[:, 1]):.4f}')
print(f'ROC-AUC test = {roc_auc_score(y_test, y_test_grid[:, 1]):.4f}')

ROC-AUC train = 0.8694
ROC-AUC test = 0.8687


In [4]:
# посмотрим набор лучших параметров
clf.best_params_

{'C': 1,
 'class_weight': 'balanced',
 'max_iter': 1000,
 'penalty': 'l2',
 'random_state': 42,
 'solver': 'saga'}

In [5]:
# лучшая обученная модель
clf.best_estimator_

LogisticRegression(C=1, class_weight='balanced', max_iter=1000, random_state=42,
                   solver='saga')

In [9]:
# список версий библиотек, используемых для проекта
!pip list

Package                       Version
----------------------------- --------------------
alabaster                     0.7.12
anaconda-client               1.11.0
anaconda-navigator            2.3.1
anaconda-project              0.11.1
anyio                         3.5.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arrow                         1.2.2
astroid                       2.11.7
astropy                       5.1
atomicwrites                  1.4.0
attrs                         21.4.0
Automat                       20.2.0
autopep8                      1.6.0
Babel                         2.9.1
backcall                      0.2.0
backports.functools-lru-cache 1.6.4
backports.tempfile            1.0
backports.weakref             1.0.post1
bcrypt                        3.2.0
beautifulsoup4                4.11.1
binaryornot                   0.4.4
bitarray                      2.5.1
bkcharts                      0.2
blac

In [ ]:
# рассмотрим распределение целевой переменной y
sns.set(rc={'figure.figsize': (9, 6)})
sns.kdeplot(df.y, fill=True, color='teal');

Отметим, что распределение ненормальное, есть длинных хвост, поэтому стоит аккуратно подходить к выбору метрики. Больше всего будем ориентироваться на MAE, так как метрику устойчива к выбросам.

# Pipeline + Grid Search

In [ ]:
COLORS = ['#408FB0', '#50B985', '#DB565F', '#FCB06E']
customPalette = sns.set_palette(sns.color_palette(COLORS))

In [ ]:
sns.boxplot(data=df, 
            x='age_bins', 
            y='charges', 
            hue='sex', 
            palette=customPalette, 
            ax=axes[0])

In [3]:
# функция для коэффициента детерминации множественной регрессии
def r2_adjusted(y_true: pd.Series, y_pred: pd.Series,
                X_test: pd.DataFrame) -> float:
    """
    Коэффициент детерминации для множественной регрессии
    """
    n = len(y_true)
    h = X_test.shape[1]
    r2 = r2_score(y_true, y_pred)

    return 1 - (1 - r2) * (n - 1) / (n - h - 1)

In [ ]:
# функция для получения метрик
def get_metrics(y_test, model, X_test, name):
    """Получение датасета с метриками"""
    y_pred = model.predict(X_test)
    
    df_metrics = pd.DataFrame()
    df_metrics['model'] = [name]
    df_metrics['MAE'] = [mean_absolute_error(y_test, y_pred)]
    df_metrics['MSE'] = [mean_squared_error(y_test, y_pred)]
    df_metrics['RMSE'] = [np.sqrt(mean_squared_error(y_test, y_pred))]
    df_metrcis['R2 adjusted'] = [r2_adjusted(y_test, y_pred, X_test)]
    return df_metrics


# metrics_train = get_metrics(y_train, pipe, X_train, 'KNN train')
# metrics_test = get_metrics(y_test, pipe, X_test, 'KNN test')

# metrics = pd.concat([metrics_train, metrics_test])
# round(metrics.set_index('model'), 2)

In [ ]:
# пример пайплайна
pipe = Pipeline(steps=[('minmax', MinMaxScaler()), 
                       ('knn', KNeighborsRegressor())])

In [ ]:
# создадим словарь с параметрами, которые хотим установить и перебрать
parameters = {
    'knn__n_neighbors': list(range(3, 32, 2)),
    'knn__metric': ['euclidean', 'manhattan', 'minkowski'],
    'knn__algorithm': ["auto", "ball_tree", "kd_tree", "brute"]
}

# подаем на вход модель, словарь с параметрами, установим, на какую метрику ориентироваться
clf = GridSearchCV(estimator=pipe,
                   param_grid=parameters,
                   scoring='neg_mean_absolute_error',
                   cv=3,
                   verbose=0)
clf.fit(X_train, y_train)

In [ ]:
# Наши параметры подобрались, посмотрим на лучшие из значений
clf.best_estimator_

In [ ]:
# Посмотрим на распределения предсказанных значений и тестовых данных.
y_pred_test = clf.predict(X_test)
sns.kdeplot(y_test, fill=True, palette=customPalette, label='y_test')
sns.kdeplot(y_pred_test, fill=True, palette=customPalette, label='y_test_pred')
plt.legend();

In [ ]:
# после того как модель обучена найдем самые важные признаки
from sklearn.inspection import permutation_importance

results = permutation_importance(clf,
                                 X_test,
                                 y_test,
                                 n_repeats=30,
                                 random_state=RANDOM_STATE,
                                 scoring='neg_mean_squared_error')

perm_df = pd.DataFrame({'col': X.columns, 'value': results.importances_mean})
perm_df = round(perm_df.sort_values(by='value', ascending=False), 1)
perm_df

In [ ]:
# изобразим графически все важные признаки
b = sns.barplot(data=perm_df, y='col', x='value', palette='viridis')
b.set_xlabel('Importances mean', fontsize=16)
b.set_ylabel('Features', fontsize=16)
b.set_title('Feature Importance', fontsize=20)
b.tick_params(labelsize=14);